In [1]:
import pandas as pd

# Load data
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv')

In [2]:
train_data.head(2)

,id,현재가,전일비,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,label
0,0,2351.703,116.656,500.0,832.124,35392.0,1.658,2706392.610,19.116,6.5,0
1,1,11687.402,51.515,0.0,304.111,2600.0,0.490,50847.441,NaN,NaN,0


In [3]:
# Check data
print(train_data.head())
print(test_data.head())
print(sample_submission.head())

   id        현재가      전일비    액면가     시가총액      상장주식수  외국인비율          거래량  \
0   0   2351.703  116.656  500.0  832.124  35392.000  1.658  2706392.610   
1   1  11687.402   51.515    0.0  304.111   2600.000  0.490    50847.441   
2   2   3100.000    0.000  500.0  969.000  31257.000  0.370        0.000   
3   3   9408.961  254.445    0.0  292.195   3105.573  1.905   132966.463   
4   4   2226.067  114.968  100.0  605.343  27191.000  2.551   522215.695   

      PER    ROE  label  
0  19.116   6.50      0  
1     NaN    NaN      0  
2   7.088  18.71      1  
3     NaN    NaN      0  
4     NaN    NaN      2  
   id        현재가      전일비     액면가       시가총액     상장주식수  외국인비율         거래량  \
0   0  65198.863  304.482     0.0   1212.443    1860.0  0.081    4793.170   
1   1   6406.595  171.084   500.0   3711.403   57931.0  1.209  282689.080   
2   2   4555.651  217.911   500.0   1596.002   35038.0  5.360  214691.924   
3   3  25048.754  490.409  5000.0  26874.700  107291.0  4.048  418903.868   
4 

In [4]:

# Preprocessing

# Check missing values
# print(train_data.isnull().sum())
# print(test_data.isnull().sum())

# Drop missing values
# train_data = train_data.dropna()
# test_data = test_data.dropna()

# 필요시 결측치 처리 (예: 평균값으로 채우기)
train_data.fillna(train_data.mean(), inplace=True)
test_data.fillna(test_data.mean(), inplace=True)

# Check missing values again
print(train_data.isnull().sum())
print(test_data.isnull().sum())

id       0
현재가      0
전일비      0
액면가      0
시가총액     0
상장주식수    0
외국인비율    0
거래량      0
PER      0
ROE      0
label    0
dtype: int64
id       0
현재가      0
전일비      0
액면가      0
시가총액     0
상장주식수    0
외국인비율    0
거래량      0
PER      0
ROE      0
dtype: int64


In [5]:
# Feature and Lable 분리
train_features = train_data.drop(columns=['label'], axis=1) # label은 예측할 값이라고 가정
train_labels = train_data['label']

In [6]:
# Data Normalization
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_data)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_features_scaled, train_labels, test_size=0.2, random_state=42)

## Step3 모델 선택 및 학습
Random Forest 모델을 사용해 예측 진행 

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# # Cross-validation
# cv_scores = cross_val_score(model, train_features_scaled, train_labels, cv=5, scoring='neg_mean_absolute_error')

# print(f'Cross-validated MSE: {-cv_scores.mean()}')

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.88      0.73       274
           1       1.00      1.00      1.00        26
           2       0.57      0.23      0.32       190

    accuracy                           0.63       490
   macro avg       0.73      0.70      0.68       490
weighted avg       0.62      0.63      0.59       490



In [9]:
from sklearn.metrics import precision_score , recall_score , confusion_matrix, classification_report

y_true = [0, 0, 1, 1, 2, 2, 2]
y_pred = [0, 1, 1, 2, 2, 2, 1]
print(confusion_matrix(y_true, y_pred))

precision = precision_score(y_true, y_pred,average= "macro")
recall = recall_score(y_true, y_pred,average= "macro")
print('정밀도: {0:.4f}, 재현율: {1:.4f}'.format(precision, recall))
print(classification_report(y_true, y_pred))

[[1 1 0]
 [0 1 1]
 [0 1 2]]
정밀도: 0.6667, 재현율: 0.5556
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.33      0.50      0.40         2
           2       0.67      0.67      0.67         3

    accuracy                           0.57         7
   macro avg       0.67      0.56      0.58         7
weighted avg       0.67      0.57      0.59         7



In [10]:
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score

# Initialize the model
model = SVR(kernel='rbf', C=1e3, gamma=0.1)

# Cross-validation
cv_scores = cross_val_score(model, train_features_scaled, train_labels, cv=5, scoring='neg_mean_absolute_error')

print(f'Cross-validated MSE: {-cv_scores.mean()}')

Cross-validated MSE: 1.0583767451723924


In [13]:
# Fit the SVR model
model.fit(train_features_scaled, train_labels)

# Predict the test data
predictions = model.predict(test_features_scaled)


In [14]:
# 필요시 test_data를 sample_submission의 길이에 맞춰 조정
if len(test_data) != len(predictions):
  test_data = test_data.head(len(sample_submission))

In [15]:
# 데이터 정규화 (변경된 test_data를 사용)
test_features_scaled = scaler.transform(test_data)

# Train the model
model.fit(train_features_scaled, train_labels)

# Predict the test data
predictions = model.predict(test_features_scaled)

In [16]:
# Save the predictions to a CSV file
sample_submission['label'] = predictions

In [17]:
# Save the predictions to a CSV file
sample_submission['label'] = predictions
sample_submission.to_csv('./data/submission.csv', index=False)

In [18]:
# MAE

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_features_scaled, train_labels, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
val_predictions = model.predict(X_val)

mae = mean_absolute_error(y_val, val_predictions)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 1.0245163908526458
